In [2]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

In [8]:
img=cv.imread('image/building1.jpg')
img_original=np.copy(img)
img_show=np.copy(img)
img_part=None

#브러시 사이즈 및 색 설정
BrushSiz=9
Color=(255,0,0)
pt1,pt2=(0,0),(0,0)
drawing=False

#그림을 그리는 함수
def paint(event,x,y,flags,param):
    global pt1, pt2, drawing, img_show, img_original, img_part

    if event==cv.EVENT_LBUTTONDOWN:
        drawing = True
        pt1=(x,y)
        img_show = np.copy(img_original)

    elif event==cv.EVENT_MOUSEMOVE and drawing:
        temp_img = np.copy(img_original)
        cv.rectangle(temp_img, pt1, (x, y), Color, 2)
        img_show = temp_img
        cv.imshow('Paint', img_show)

    elif event==cv.EVENT_LBUTTONUP:
        drawing = False
        pt2=(x,y)

        # 최종 사각형 그리기
        cv.rectangle(img_show, pt1, pt2, Color, 2)
        print("pt1 좌표:",pt1)
        print("pt2 좌표:",pt2)
        cv.imshow('Paint', img_show)

        #선택한 좌표 기반 이미지 준비
        x1, y1 = pt1[0], pt1[1]
        x2, y2 = pt2[0], pt2[1]

        #시작적 종료점 확인
        x_min, x_max = min(x1, x2), max(x1, x2)
        y_min, y_max = min(y1, y2), max(y1, y2)
    
        # 이미지 자르기
        img_part = img[y_min:y_max, x_min:x_max]

        #자른 이미지 표시
        cv.imshow('Cropped Image', img_part)
        
    
    cv.imshow('Paint',img_show)
cv.namedWindow('Paint')
cv.setMouseCallback('Paint',paint)

while(True):
    if cv.waitKey(1)==ord('q'):
        break

cv.destroyAllWindows()

gray1=cv.cvtColor(img_part,cv.COLOR_BGR2GRAY)
gray2=cv.cvtColor(img_original,cv.COLOR_BGR2GRAY)

sift=cv.SIFT_create()
kp1,des1=sift.detectAndCompute(gray1,None)
kp2,des2=sift.detectAndCompute(gray2,None)

flann_matcher=cv.DescriptorMatcher_create(cv.DescriptorMatcher_FLANNBASED)
knn_match=flann_matcher.knnMatch(des1,des2,2)

T=0.7
good_match=[]
for nearest1,nearest2 in knn_match:
    if (nearest1.distance/nearest2.distance)<T:
        good_match.append(nearest1)

points1=np.float32([kp1[gm.queryIdx].pt for gm in good_match])
points2=np.float32([kp2[gm.trainIdx].pt for gm in good_match])
H,_=cv.findHomography(points1,points2,cv.RANSAC)

h1,w1=img_part.shape[0],img_part.shape[1]
h2,w2=img_original.shape[0],img_original.shape[1]

box1=np.float32([[0,0],[0,h1-1],[w1-1,h1-1],[w1-1,0]]).reshape(4,1,2)
box2=cv.perspectiveTransform(box1,H)

img_part=cv.polylines(img_part,[np.int32(box2)],True,(0,255,0),8)

img_match=np.empty((max(h1,h2),w1+w2,3),dtype=np.uint8)
cv.drawMatches(img_part,kp1,img_original,kp2,good_match,img_match,flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

cv.imshow('homography', img_match)
k=cv.waitKey()
cv.destroyAllWindows()

pt1 좌표: (607, 288)
pt2 좌표: (911, 516)
